In [8]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv

# ChromeDriver 路径
driver_path = r"C:\chromedriver-win64\chromedriver.exe"
options = Options()
options.add_argument('--incognito')

# 初始化 WebDriver
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=options)

# 打开目标网页
url = "https://www.85cafe.com/storeslist.php"
driver.get(url)

# 等待页面加载
time.sleep(3)

# 存储数据
store_data = []

# 点击“搜尋”按钮进行查询
submit_button = driver.find_element(By.CSS_SELECTOR, "input[type='submit']")
submit_button.click()
time.sleep(3)  # 等待页面加载

# 爬取所有门市
try:
    # 获取页面所有门市的基础数据
    while True:
        store_list = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "tbody tr"))
        )

        for store in store_list:
            store_name = store.find_element(By.CSS_SELECTOR, "td:nth-child(1) a").text.strip()
            address = store.find_element(By.CSS_SELECTOR, "td:nth-child(2) a").text.strip()
            store_phone = store.find_element(By.CSS_SELECTOR, "td:nth-child(3) a").text.strip()
            business_hours = store.find_element(By.CSS_SELECTOR, "td:nth-child(4) a").text.strip()

            # 存储抓取到的数据
            store_data.append([store_name, store_phone, address, business_hours, ""])
            # 打印出结果
            print(f"门市：{store_name}, 地址：{address}, 电话：{store_phone}, 营业时间：{business_hours}")

        # 向下滚动页面以加载更多数据
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)

        # 检查是否能加载更多数据（如果没有新的数据加载，停止）
        new_store_list = driver.find_elements(By.CSS_SELECTOR, "tbody tr")
        if len(new_store_list) == len(store_data):
            break

except Exception as e:
    print(f"爬取过程中发生错误：{e}")

# 点击筛选并提交搜索
def apply_filter_and_search(checkbox_name, special_service):
    # 选择筛选框
    checkbox = driver.find_element(By.NAME, checkbox_name)
    checkbox.click()
    time.sleep(1)
    
    # 提交搜索表单
    submit_button = driver.find_element(By.CSS_SELECTOR, "input[type='submit']")
    submit_button.click()
    time.sleep(3)  # 等待页面加载

    # 获取筛选后的门市数据
    filtered_stores = []

    try:
        while True:
            store_list = WebDriverWait(driver, 20).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "tbody tr"))
            )

            for store in store_list:
                store_name = store.find_element(By.CSS_SELECTOR, "td:nth-child(1) a").text.strip()
                address = store.find_element(By.CSS_SELECTOR, "td:nth-child(2) a").text.strip()
                store_phone = store.find_element(By.CSS_SELECTOR, "td:nth-child(3) a").text.strip()
                business_hours = store.find_element(By.CSS_SELECTOR, "td:nth-child(4) a").text.strip()

                # 将特殊服务信息添加到数据
                filtered_stores.append([store_name, store_phone, address, business_hours, special_service])
                # 打印筛选结果
                print(f"门市：{store_name}, 地址：{address}, 电话：{store_phone}, 营业时间：{business_hours}, 特殊服务：{special_service}")

            # 向下滚动页面以加载更多数据
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(3)

            # 检查是否能加载更多数据（如果没有新的数据加载，停止）
            new_store_list = driver.find_elements(By.CSS_SELECTOR, "tbody tr")
            if len(new_store_list) == len(filtered_stores):
                break

    except Exception as e:
        print(f"筛选 {checkbox_name} 过程中发生错误：{e}")

    return filtered_stores

# 先抓取所有门市
driver.get(url)  # 重新加载页面
time.sleep(3)

# 获取24小时营业门市
stores_24h = apply_filter_and_search('24house', '24小时营业门市')

# 获取麵包烘培門市
driver.get(url)  # 重新加载页面
time.sleep(3)
stores_bakery = apply_filter_and_search('bakery', '麵包烘培門市')

# 合并数据：先加入全部门市，再加入筛选后的门市
final_store_data = store_data + stores_24h + stores_bakery

# 保存到 CSV 文件
with open("85cafe_store.csv", "w", newline="", encoding="utf-8-sig") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["门市名称", "电话", "地址", "营业时间", "特殊服务"])
    writer.writerows(final_store_data)

# 关闭浏览器
driver.quit()

print("爬取和保存数据完成")


爬取过程中发生错误：Message: no such element: Unable to locate element: {"method":"css selector","selector":"td:nth-child(1) a"}
  (Session info: chrome=131.0.6778.265); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6BDAA6CF5+28821]
	(No symbol) [0x00007FF6BDA13880]
	(No symbol) [0x00007FF6BD8B578A]
	(No symbol) [0x00007FF6BD9091BE]
	(No symbol) [0x00007FF6BD9094AC]
	(No symbol) [0x00007FF6BD8FC52C]
	(No symbol) [0x00007FF6BD92F33F]
	(No symbol) [0x00007FF6BD8FC3F6]
	(No symbol) [0x00007FF6BD92F510]
	(No symbol) [0x00007FF6BD94F412]
	(No symbol) [0x00007FF6BD92F0A3]
	(No symbol) [0x00007FF6BD8FA778]
	(No symbol) [0x00007FF6BD8FB8E1]
	GetHandleVerifier [0x00007FF6BDDDFCED+3408013]
	GetHandleVerifier [0x00007FF6BDDF745F+3504127]
	GetHandleVerifier [0x00007FF6BDDEB63D+3455453]
	GetHandleVerifier [0x00007FF6BDB6BDFB+835995]
	(No symbol) [0x00007FF6BDA1EB9F]
	(No

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv

# ChromeDriver 路徑
driver_path = r"C:\chromedriver-win64\chromedriver.exe"
options = Options()
options.add_argument('--incognito')

# 初始化 WebDriver
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=options)

# 打開目標網頁
url = "https://www.85cafe.com/storeslist.php"
driver.get(url)

# 等待頁面加載
time.sleep(3)

try:
    # 點擊提交按鈕
    submit_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "input.SR_searchBTN"))
    )
    submit_button.click()

    # 等待結果頁加載
    time.sleep(3)

    # 滾動頁面以確保所有數據加載完成
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # 滾動到底部
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # 等待頁面加載

        # 檢查是否有更多內容
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break  # 如果高度未變化，說明沒有更多內容
        last_height = new_height

    # 抓取門市資料表格的所有行
    rows = WebDriverWait(driver, 20).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "tbody tr"))
    )

    # 總門市數量
    total_stores = len(rows)
    print(f"網站顯示的總門市數量：{total_stores}")

    # 存儲資料
    store_data = []
    successful_count = 0  # 成功抓取的門市數量

    for row in rows:
        try:
            # 增加顯式等待，確保元素可見
            store_name = WebDriverWait(row, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "td:nth-child(1) a"))
            ).text.strip()

            address = row.find_element(By.CSS_SELECTOR, "td:nth-child(2) a").text.strip()
            phone = row.find_element(By.CSS_SELECTOR, "td:nth-child(3) a").text.strip()
            business_hours = row.find_element(By.CSS_SELECTOR, "td:nth-child(4) a").text.strip()

            # 輸出結果到終端
            print(f"門市名稱: {store_name}")
            print(f"地址: {address}")
            print(f"電話: {phone}")
            print(f"營業時間: {business_hours}")
            print("=" * 40)

            # 添加到資料列表
            store_data.append([store_name, address, phone, business_hours])
            successful_count += 1  # 成功抓取數量+1

        except Exception as e:
            print(f"抓取資料時出現問題: {e}")

    print(f"最終成功抓取的門市數量：{successful_count}")

    # 將資料存儲到 CSV 文件
    with open("85c_stores.csv", "w", newline="", encoding="utf-8-sig") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["門市名稱", "地址", "電話", "營業時間"])
        writer.writerows(store_data)

    print("資料已成功存儲到 85c_stores.csv")

except Exception as e:
    print(f"爬取過程中發生錯誤: {e}")

finally:
    driver.quit()
